# Hairsalon  Database


## About this notebook

This notebook was created to help with the initial analysis of the hair salon database. It conatains some functions that will help tha analysis more easy and productive.


In [59]:
ROOT_PATH = '/home/host'
DATA_PATH = "%s/data/raw" % ROOT_PATH
DB_FILE = "%s/hairsalon.db" % DATA_PATH
import sqlite3 as sql3
db=sql3.connect(DB_FILE)
import pandas as pd

In [60]:
##############################################################################
##########                DATABASE FUNCTIONS                     #############
##############################################################################

#### List the name of the tables on the database 
def listTables():
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    cur = db.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    res = cur.fetchall()
    cur.close()
    db.close()
    return(res)

#### Read function to import data from the SQL to a pandas dataframe.
def readSQL(query):
    import pandas as pd
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    df = pd.read_sql_query(query, db)
    db.close()
    return(df)

#### Write a pandas dataframe into an SQL table. Use overwrite=True if you want to delete 
#### first a pre-existent table with the same name. Use append=True if you want to append
#### the data in the dataframe to a pre-existent table.
def writeSQL(df,tablename,overwrite=False, append=False):
    import pandas as pd
    import sqlite3 as sql
    db = sql.connect(DB_FILE)
    if (overwrite):
        action = "replace"
    elif (append):
        action = "append"
    else: 
        action = "fail"
    df.to_sql(tablename, db, if_exists=action)
    db.close()

#### Generate a view based on the provided SQL query
def createView(viewname, query):
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    view = 'CREATE VIEW {} AS {}'.format(viewname,query)
    #view = """CREATE VIEW ? AS 
    #?"""
    cur = db.cursor()
    #cur.execute(view, viewname, query)
    cur.execute(view)
    cur.close()
    db.close()

def execquery (query):
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    cur = db.cursor()
    cur.execute(query)
    cur.close()
    db.close()
     

<h1> import from csv data into sqltables to run once recreates all tables </h1>

In [61]:
import pandas as pd
file_tuple=("Client Cancellations0","Future Bookings (All Clients)0","No-Show Report0",
            "Product Listing (Retail)0","Receipt Transactions0","Service Listing0")
table_tuple=("cancellation","futurebooking","noshow","product","receipt","service")
tableiter=iter(table_tuple)

#loop over csv files and crate a correpsonding table for each file 
for f in file_tuple:
    file=f'{DATA_PATH}/{f}.csv'
    df=pd.read_csv(file)
    #rename columns remove spaces and lower allcharacters
    df.rename(columns=lambda x:x.replace(" ",""),inplace=True)
    df.rename(columns=lambda x:x.lower(),inplace=True)
    df.to_sql(next(tableiter),db,if_exists='replace',index=False)

In [62]:
db.commit()

In [63]:
cancellations = readSQL('''SELECT * FROM cancellation''')
cancellations.head()

,canceldate,code,service,staff,bookingdate,canceledby,days
0,05/22/2018,MASA01,CFC,JOANNE,05/24/2018,ADMIN,2.0
1,06/27/2018,MASA01,CBAL,KELLY,06/27/2018,ADMIN,0.0
2,07/03/2018,MASA01,SHCM,BECKY,07/10/2018,BECKY,7.0
3,07/10/2018,MASA01,SHCC,JJ,07/11/2018,KELLY,1.0
4,07/24/2018,TRUA01,SHCM,KELLY,07/24/2018,TANYA,0.0


In [64]:
futurebookings=readSQL('''SELECT * FROM futurebooking''')
futurebookings.head()

,code,staff,service,date,time,timeint
0,JUNJ01,JJ,CON,03/14/2018,3:50:00 PM,950
1,NORJ01,JJ,SHCM,03/14/2018,4:00:00 PM,960
2,CORS01,JJ,SHCW,03/15/2018,2:10:00 PM,850
3,KERT01,JJ,SHCW,03/15/2018,10:00:00 AM,600
4,LEHJ01,JJ,SHCM,03/15/2018,6:00:00 PM,1080


In [65]:
receipts=readSQL('''SELECT * FROM receipt''')
receipts.head()

,receipt,date,description,client,staff,quantity,amount,gst,pst
0,10,03/15/2018,Women's hair cut,KERT01,JJ,1,82.0,4.10,6.56
1,100,04/05/2018,Women's hair cut,COOM01,SINEAD,1,70.0,3.50,5.60
2,1000,06/09/2018,Consultation,PEDM01,JJ,1,0.0,0.00,0.00
3,1001,06/09/2018,Women's hair cut,BAIS01,KELLY,1,55.0,2.75,4.40
4,1003,06/09/2018,Women's hair cut,FRAL01,KELLY,1,65.0,3.25,5.20


In [66]:
noshows=readSQL('''SELECT * FROM noshow''')
noshows.head()

,date,code,service,staff
0,04/13/2018,None,SHCW,JJ
1,07/27/2018,AMAB01,SHCM,KELLY
2,06/07/2018,APPS01,SBD,KELLY
3,06/05/2018,ASIS01,SHCW,KELLY
4,05/31/2018,BISD01,CON,SINEAD


In [67]:
products=readSQL('''SELECT * FROM product''')
products.head()

,isactive,code,description,supplier,brand,category,price,onhand,minimum,maximum,cost,cog,ytd,package
0,1,AGBALANCE335M,AG balance 355ml,VENUS,AG,RETAIL,30.0,2,2,2,0.0,0.0,27.0,0
1,1,AGBOOST355ML,AG Boost 355ml,VENUS,AG,RETAIL,30.0,3,2,2,18.0,54.0,0.0,0
2,1,AGCLOUD107ML,AG Cloud 107ml,VENUS,AG,RETAIL,26.0,4,2,2,15.6,62.4,23.4,0
3,1,AGDRYLIFT44ML,AG Dry Lift 44ml,VENUS,AG,RETAIL,24.0,1,2,2,14.4,14.4,43.2,0
4,1,AGREMEDY148ML,AG Remedy 148ml,VENUS,AG,RETAIL,26.0,3,2,2,15.6,46.8,0.0,0


In [68]:
services=readSQL('''SELECT * FROM service''')
services.head()

,isactive,code,desc,cate,price,cost
0,1,CAL,accent lights,COLOR,0,10
1,1,CALC,Accent lights and color,COLOR,0,10
2,1,CAS,Color additional service,COLOR,0,10
3,1,CBAL,Balayage,COLOR,0,10
4,1,CCAMO,Men's camo color,COLOR,45,5


In [69]:
#SQLITE DB date and time are saved as text values - the ocde below updates the format in the DB
#converts format date MM/DD/YYYY =>ISO FORMAT 8601 YYYY-DD-MM
#converts format date 3:50:0 PM =>ISO FORMAT HH:MM:SS
import datetime as dt
cancellations['canceldate']=pd.to_datetime(cancellations['canceldate'],format="%m/%d/%Y")
cancellations['canceldate'] = cancellations['canceldate'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
cancellations['bookingdate']=pd.to_datetime(cancellations['bookingdate'],format="%m/%d/%Y")
cancellations['bookingdate'] = cancellations['bookingdate'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
cancellations.to_sql('cancellation',db,if_exists='replace',index=False)

futurebookings['date']=pd.to_datetime(futurebookings['date'],format="%m/%d/%Y")
futurebookings['date'] = futurebookings['date'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
futurebookings['time']=pd.to_datetime(futurebookings['time'],infer_datetime_format=True)
futurebookings['time'] = futurebookings['time'].apply(lambda x: dt.datetime.strftime(x, '%H:%M:%S'))
futurebookings.to_sql('futurebooking',db,if_exists='replace',index=False)

receipts['date']=pd.to_datetime(receipts['date'],format="%m/%d/%Y")
receipts['date'] = receipts['date'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
receipts.to_sql('receipt',db,if_exists='replace',index=False)

noshows['date']=pd.to_datetime(noshows['date'],format="%m/%d/%Y")
noshows['date'] = noshows['date'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))
noshows.to_sql('noshow',db,if_exists='replace',index=False)

In [73]:
query = """SELECT MIN(canceldate), MAX(canceldate), MIN(bookingdate), Max (bookingdate),min(days),max(days),count(*)
         FROM cancellation"""
resquery = readSQL(query)
resquery

,MIN(canceldate),MAX(canceldate),MIN(bookingdate),Max (bookingdate),min(days),max(days),count(*)
0,2018-03-27,2018-07-28,2018-03-29,2018-09-21,-37.0,66.0,243


In [74]:
query = """SELECT MIN(date), MAX(date), count(*)
         FROM noshow"""
resquery = readSQL(query)
resquery

,MIN(date),MAX(date),count(*)
0,2018-03-14,2018-07-29,60


In [75]:
query = """SELECT MIN(date), MAX(date), count(*)
         FROM futurebooking"""
resquery = readSQL(query)
resquery

,MIN(date),MAX(date),count(*)
0,2018-03-14,2019-02-15,1906


In [7]:
customers = readSQL('''SELECT * FROM customers LIMIT 100''')
orders_items=readSQL('''SELECT * FROM ORDER_ITEMS LIMIT 100''')

In [8]:
#customers.head()
orders_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [9]:
writeSQL(orders,"test1")

In [10]:
query = """SELECT a.customer_id, a.customer_city, a.customer_zip_code_prefix,
         b.order_id ,b.order_status, b.order_delivered_customer_date
         FROM customers a 
         INNER JOIN orders b ON a.customer_id = b.customer_id"""

test2 = readSQL(query)
test2.head()


,customer_id,customer_city,customer_zip_code_prefix,order_id,order_status,order_delivered_customer_date
0,9ef432eb6251297304e76186b10a928d,sao paulo,3149,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-10 21:25:13
1,b0830fb4747a6c6d20dea0b8c802d7ef,barreiras,47813,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-08-07 15:27:45
2,41ce2a54c0b03bf3443c3d931a367089,vianopolis,75265,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-17 18:06:29
3,f88197465ea7920adcdbec7375364d82,sao goncalo do amarante,59296,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-12-02 00:28:42
4,8ab97904e6daea8866dbdbc4fb7aad2c,santo andre,9195,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-16 18:17:02


In [14]:

createView("test1_v",query)

In [15]:
import sqlite3 as sql3

db = sql3.connect(DB_FILE)
cur = db.cursor()
cur.execute("DROP VIEW test1_v")
cur.close()
db.close()


In [16]:
db = sql3.connect(DB_FILE)
cur = db.cursor()
cur.execute("DROP TABLE test1")
cur.close()
db.close()

In [17]:
aa = readSQL("SELECT * FROM closed_deals where declared_product_catalog_size <> ''")
aa.head()


,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,bae38661a27f6228ba38c36e766ed769,6a6b1614baaaf766293c17d8cb8c5a9d,f42a2bd194f7802ab052a815c8de65b7,495d4e95a8cf8bbf8b432b612a2aa328,2018-09-10 14:18:01,stationery,industry,cat,,,20-50,reseller,2000,100000
1,72e81de94dfc0373b006ca75e9c851a1,2f1e89e231b096f4ff1f55dfe8607a7b,e7dff61b78bebffa71678e126ce669ad,068066e24f0c643eb1d089c7dd20cd73,2018-10-31 16:54:59,computers,online_medium,cat,True,,1-5,reseller,80,0
2,a8d2884db49e0769ad4a0b8dce2e143c,d65f74c704563bcb8baa915fbf8cfa1e,6aa3b86a83d784b05f0e37e26b20860d,de63de0d10a6012430098db33c679b0b,2018-09-27 12:31:01,health_beauty,online_medium,cat,,False,,reseller,15,20000
3,20fcec0873b39b4df3df34140d77d6e7,cee3c487c56d29554a22d112af618301,084b6fbb10729ed4da8c3d3f5a3ae7c9,0a0fb2b07d841f84fb6714e35c723075,2018-11-14 18:04:19,other,other,cat,False,False,1-5,other,120,6000
4,a55b94066766f2d29fc22a3bb52d78f6,5181ea7b0d346ed14c5c07f0ff22b2b4,fdb16d3cbbeb5798f2f66c4096be026d,4ef15afb4b2723d8f3d81e51ec7afefe,2018-09-04 14:15:04,toys,online_big,cat,,,5-20,reseller,1000,180000


Question 1: products sold per product category 

In [30]:
query = """SELECT b.product_category_name, count(*) total_prods
         FROM order_items a 
         INNER JOIN PRODUCTS b  ON a.product_id = b.product_id
         GROUP BY b.product_category_name order by total_prods DESC LIMIT 10"""

test2 = readSQL(query)
test2


,product_category_name,total
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
5,utilidades_domesticas,6964
6,relogios_presentes,5991
7,telefonia,4545
8,ferramentas_jardim,4347
9,automotivo,4235


In [31]:
query = """SELECT b.product_category_name, SUM(Price) Total
         FROM order_items a 
         INNER JOIN PRODUCTS b  ON a.product_id = b.product_id
         GROUP BY b.product_category_name order by total DESC LIMIT 10"""

test2 = readSQL(query)
test2


,product_category_name,Total
0,beleza_saude,1258681.34
1,relogios_presentes,1205005.68
2,cama_mesa_banho,1036988.68
3,esporte_lazer,988048.97
4,informatica_acessorios,911954.32
5,moveis_decoracao,729762.49
6,cool_stuff,635290.85
7,utilidades_domesticas,632248.66
8,automotivo,592720.11
9,ferramentas_jardim,485256.46


In [32]:
#Total Price Per Catagory
query = """SELECT b.product_category_name, SUM(Price) Total
         FROM order_items a 
         INNER JOIN PRODUCTS b  ON a.product_id = b.product_id
         GROUP BY b.product_category_name order by total DESC LIMIT 10"""

test2 = readSQL(query)
test2


,product_category_name,Total
0,beleza_saude,1258681.34
1,relogios_presentes,1205005.68
2,cama_mesa_banho,1036988.68
3,esporte_lazer,988048.97
4,informatica_acessorios,911954.32
5,moveis_decoracao,729762.49
6,cool_stuff,635290.85
7,utilidades_domesticas,632248.66
8,automotivo,592720.11
9,ferramentas_jardim,485256.46


In [33]:
#MEAN Price Per Catagory
query = """SELECT b.product_category_name, AVG(Price) Total
         FROM order_items a 
         INNER JOIN PRODUCTS b  ON a.product_id = b.product_id
         GROUP BY b.product_category_name order by total DESC LIMIT 10"""

test2 = readSQL(query)
test2


,product_category_name,Total
0,pcs,1098.340542
1,portateis_casa_forno_e_cafe,624.285658
2,eletrodomesticos_2,476.124958
3,agro_industria_e_comercio,342.124858
4,instrumentos_musicais,281.616000
5,eletroportateis,280.778468
6,portateis_cozinha_e_preparadores_de_alimentos,264.568667
7,telefonia_fixa,225.693182
8,construcao_ferramentas_seguranca,208.992371
9,relogios_presentes,201.135984


In [9]:
## Calculate distance in meters between two geographical positions
def geoDistance(lat1,lon1, lat2,lon2):
  import math
  R = 6.371e6  ## meters
  lat1 = math.radians(lat1)
  lat2 = math.radians(lat2)
  delta1 = math.radians(lat2-lat1)
  delta2 = math.radians(lon2-lon1)
  a = (math.sin(delta1/2)*math.sin(delta1/2)) + (math.cos(lat1)*math.cos(lat2)*(math.sin(delta2/2)*math.sin(delta2/2)))
  c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
  return(R * c)

geoDistance(-23.5,-46.6,-22.0, -45.0)


164079.01370941926

NB Customers Per Item Category

In [4]:
query1="""SELECT oi.*, p.*
        FROM ORDER_ITEMS oi
        INNER JOIN PRODUCTS p ON oi.product_id=p.product_id"""
createView("ORDER_ITEMS_PRODUCTS_v",query1)


query2="""SELECT c.*, o.*
        FROM CUSTOMERS c
        INNER JOIN ORDERS o ON o.customer_id=c.customer_id"""
createView("ORDERS_CUSTOMERS_v",query2)

OperationalError: table ORDER_ITEMS_PRODUCTS_v already exists

In [5]:
query3 = """SELECT v1.product_category_name, COUNT(distinct v2.customer_id) total_customer
         FROM  ORDER_ITEMS_PRODUCTS_v v1
         INNER JOIN ORDERS_CUSTOMERS_v v2 ON v1.order_id = v2.order_id
         GROUP BY v1.product_category_name Order BY total_customer DESC """
test2 = readSQL(query3)
test2.head()

,product_category_name,total_customer
0,cama_mesa_banho,9417
1,beleza_saude,8836
2,esporte_lazer,7720
3,informatica_acessorios,6689
4,moveis_decoracao,6449


In [6]:
#Sellers and Geolocation
query5="""SELECT s.*,g.*  from sellers s
LEFT JOIN geolocation g on s.seller_zip_code_prefix=g.geolocation_zip_code_prefix"""
test5 = readSQL(query5)
test5.head()
createView("sellers_v",query5)

OperationalError: table sellers_v already exists

In [30]:
query51="""pragma table_info(orders)"""
test51=readSQL(query51)
print(test51)

   cid                           name  type  notnull dflt_value  pk
0    0                       order_id  text        1       None   0
1    1                    customer_id  text        0       None   0
2    2                   order_status  text        0       None   0
3    3       order_purchase_timestamp  text        0       None   0
4    4              order_approved_at  text        0       None   0
5    5   order_delivered_carrier_date  text        0       None   0
6    6  order_delivered_customer_date  text        0       None   0
7    7  order_estimated_delivery_date  text        0       None   0


In [18]:
query6="""select oi.seller_id,SUM(price) total,avg(price)
from order_items oi
inner join sellers s on oi.seller_id=s.seller_id
group by oi.seller_id order by total desc limit 10"""
test6=readSQL(query6)
print(test6)

DatabaseError: Execution failed on sql 'select oi.seller_id,SUM(price) total,avg(price)
from order_items oi
inner join sellers s on oi.seller_id=s.seller_id
group by oi.seller_id order by total desc limit 10': no such table: order_items

In [65]:
import pandas as pd
sellertimeframedf.MAX= pd.to_datetime(sellertimeframedf.MAX)
sellertimeframedf.MIN=pd.to_datetime(sellertimeframedf.MIN)
sellertimeframedf.dtypes

seller_id            object
nborders              int64
MIN          datetime64[ns]
MAX          datetime64[ns]
dtype: object

In [83]:
import numpy as np
sellertimeframedf['month']= ((sellertimeframedf.MAX-sellertimeframedf.MIN)/np.timedelta64(1, 'M')).astype('int')

In [84]:
sellertimeframedf.head()

,seller_id,nborders,MIN,MAX,month
0,6560211a19b47992c3666cc44a7e94c0,2033,2017-02-17 07:39:19,2018-08-29 09:14:11,18
1,4a3ca9315b744ce9f8e9374361493884,1987,2017-01-08 09:35:14,2018-08-27 10:46:11,19
2,1f50f920176fa81dab994f9023523100,1931,2017-04-03 22:00:31,2018-08-25 20:10:27,16
3,cc419e0650a3c5ba77189a1882b7556a,1775,2017-01-31 17:15:33,2018-08-27 16:12:22,18
4,da8622b14eb17ae2831f4ac5b9dab84a,1551,2017-02-05 21:46:05,2018-08-28 21:56:30,18


2 months
